<table class="ee-notebook-buttons" align="center">
    <td><a target="_blank"  href="https://colab.research.google.com/github/yotarazona/scikit-eo/blob/main/examples/notebooks/02.%20Estimated%20area%20and%20uncertainty%20in%20Machine%20Learning.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# **_<div class="alert alert-success"><font color='darkred'> Tutorials: 02 Estimating area and uncertainty in Machine Learning</font></div>_**

In this example, after obtaining the predicted class map, we are in a case where we want to know the uncertainties of each class. The assessing accuracy and area estimate will be obtained following guidance proposed by [(Olofsson et al., 2014)](https://doi.org/10.1016/j.rse.2014.02.015). All that users need are the confusion matrix and a previously obtained predicted class map.

> **Keep in mind**: the most critical recommendation is that the sampling design should be a *probability sampling design*. An essential element of probability sampling is that randomization is incorporated into the sample selection protocol. Various probability sampling designs can be applied for precision assessment and area estimation, the most commonly used designs being simple random, stratified random, and systematic. Please see the workd by [Olofsson et al. (2014)](https://doi.org/10.1016/j.rse.2014.02.015) for more details.

For this particular example, the samples follow a simple random sample design.

# 1.0 Libraries

To install ```scikit-eo``` you can do it with the following line:

In [ ]:
!pip install scikeo

Libraries to be used:

In [1]:
import rasterio
import numpy as np
import pandas as pd
#from scikeo.process import confintervalML
#from scikeo.process import print_info

Reading dataset directly from the external server (GitHub in this case). To do this, ```requests```, ```zipfile``` and ```io``` will be installed before. Then you will only need to run the following line:

In [2]:
import requests, zipfile
from io import BytesIO

# Defining the zip file URL
url = 'https://github.com/yotarazona/data/raw/main/data/02_Uncertainty.zip'

# Split URL to get the file name
filename = url.split('/')[-1]

# Downloading the file by sending the request to the URL
req = requests.get(url)

# extracting the zip file contents
file = zipfile.ZipFile(BytesIO(req.content))
file.extractall()

## 2.0 Reading classified raster and confusion matrix

In [19]:
data_dir = "02_Uncertainty/"

# classified raster
path_raster = data_dir + "LC08_232066_20190727_Label.tif"
img = rasterio.open(path_raster).read(1)

# confusion matrix
path_matrix = data_dir + 'confusion_matrix.csv'
conf_error = pd.read_csv(path_matrix, index_col= 0, sep = ';')

Confusion matrix

In [20]:
conf_error

,1.0,2.0,3.0,4.0,Total,Users_Accuracy,Commission
1.0,15.0,0.00,0.0,0.000000,15.0,100.000000,0.000000
2.0,0.0,15.00,0.0,1.000000,16.0,93.750000,6.250000
3.0,0.0,0.00,14.0,0.000000,14.0,100.000000,0.000000
4.0,0.0,1.00,0.0,18.000000,19.0,94.736842,5.263158
Total,15.0,16.00,14.0,19.000000,NaN,NaN,NaN
Producer_Accuracy,100.0,93.75,100.0,94.736842,NaN,NaN,NaN
Omission,0.0,6.25,0.0,5.263158,NaN,NaN,NaN


Only confusion matrix values:

In [27]:
values = conf_error.iloc[0:4, 0:4].to_numpy()
values

array([[15.,  0.,  0.,  0.],
       [ 0., 15.,  0.,  1.],
       [ 0.,  0., 14.,  0.],
       [ 0.,  1.,  0., 18.]])

## 3.0 Results

Obtaining estimated area and uncertainty. Be careful with the parameter to be used with the ```confintervalML()``` function. Let's explain in detail:
- *matrix*: is the confusion matrix with only values
- *image_pred*: the classified image in rows and cols (2d)
- *pixel_size*: the pixel size
- *nodata*: in this image nodata has a -9999 value, but in other cases it could take 0 or NaN. So, be careful with this parameter.

In [25]:
img = np.array([67246,79983,825123,651448])

In [28]:
result = confintervalML(matrix = values, image_pred = img, pixel_size = 30, nodata = -9999)

In [29]:
print_info(result)

***** Confusion Matrix by Estimated Proportions of area an uncertainty*****

Overall accuracy with 95%
0.975806 ± 0.041823

Confusion matrix
              1         2         3         4  Total[Wi]  Area[pixels]
1      0.041413  0.000000  0.000000  0.000000   0.041413       67246.0
2      0.000000  0.046178  0.000000  0.003079   0.049257       79983.0
3      0.000000  0.000000  0.508143  0.000000   0.508143      825123.0
4      0.000000  0.021115  0.000000  0.380072   0.401187      651448.0
Total  0.041413  0.067293  0.508143  0.383151   1.000000     1623800.0

User´s accuracy with 95%
0: 1.000000 ± 0.000000
1: 0.937500 ± 0.122500
2: 1.000000 ± 0.000000
3: 0.947368 ± 0.103158

Estimating area (Ha) and uncertainty with 95%
0: 6052.140000 ± 0.000000
1: 9834.371941 ± 6112.125597
2: 74261.070000 ± 0.000000
3: 55994.418059 ± 6112.125597


As a final comment, it is important to note that the calculation of uncertainties allows us to produce scientifically rigorous and transparent estimates of precision and area. Therefore, it is strongly recommended to obtain uncertainties for Land Use and Land Cover change analyses.

In [30]:
def confintervalML(matrix, image_pred, pixel_size = 10, conf = 1.96, nodata = None):
    
    '''
    The error matrix is a simple cross-tabulation of the class labels allocated by the classification of the remotely 
    sensed data against the reference data for the sample sites. The error matrix organizes the acquired sample data 
    in a way that summarizes key results and aids the quantification of accuracy and area. The main diagonal of the error 
    matrix highlights correct classifications while the off-diagonal elements show omission and commission errors. 
    The cell entries and marginal values of the error matrix are fundamental to both accuracy assessment and area 
    estimation. The cell entries of the population error matrix and the parameters derived from it must be estimated 
    from a sample. This function shows how to obtain a confusion matrix by estimated proportions of area with a confidence
    interval at 95% (1.96). I strongly recommend reading Olofsson et al. (2014)'s paper for more technical and scientific details 
    on the implementation of this function.
     
    Parameters:
    
        matrix: confusion matrix or error matrix in numpy.ndarray.
            
        image_pred: Could be an array with 2d (rows, cols). This array should be the image classified 
                    with predicted classes. Or, could be a list with number of pixels for each class.
            
        pixel_size: Pixel size of the image classified. By default is 10m of Sentinel-2.
            
        conf: Confidence interval. By default is 95%.
    
    Return:
        
        Information of confusion matrix by proportions of area, overall accuracy, user's accuracy with confidence interval 
        and estimated area with confidence interval as well.
         
    Reference:
    
        Olofsson, P., Foody, G.M., Herold, M., Stehman, S.V., Woodcock, C.E., and Wulder, M.A. 2014. “Good practices 
        for estimating area and assessing accuracy of land change.” Remote Sensing of Environment, Vol. 148: 42–57. 
        doi:https://doi.org/10.1016/j.rse.2014.02.015.
    
    Note:
        Columns and rows in a confusion matrix indicate reference and prediction respectively. Additionally, the most critical
        recommendation is that the sampling design should be a *probability sampling design*. An essential element of probability sampling
        is that randomization is incorporated into the sample selection protocol. Various probability sampling designs can be applied for
        precision assessment and area estimation, the most commonly used designs being simple random, stratified random, and systematic 
        (Olofsson et al., 2014). 
    '''
    
    if not isinstance(matrix, (np.ndarray)):
        raise TypeError('"matrix" must be numpy.ndarray with rows and cols.')
        
    if isinstance(image_pred, (np.ndarray)):
        
        if image_pred.ndim == 2:
            
            # xlabel
            if nodata is None:
                image_pred = image_pred.astype(float)
            else:
                image_pred = image_pred.astype(float)
                image_pred[image_pred == nodata] = np.nan
        
            # classes
            iclass = np.unique(image_pred[~np.isnan(image_pred)])
    
            # ni
            ni = np.sum(matrix, axis = 1)
    
            # number of classes
            n = matrix.shape[0]
    
            pixels = []
    
            for i in iclass:
                pixels.append((image_pred == i).sum()) #((30**2)/10000) # ha    
    
            wi = (np.array([pixels])/np.array([pixels]).sum()).flatten()
    
            pixels = np.array(pixels)
        
        elif image_pred.ndim == 1:
            
            # ni
            ni = np.sum(matrix, axis = 1)
    
            # number of classes
            n = matrix.shape[0]
            
            # classes
            iclass = np.arange(len(image_pred))
            
            # pixels
            pixels = np.array(image_pred)
            
            # pesos
            wi = (np.array([pixels])/np.array([pixels]).sum()).flatten()
    
        else:
            raise TypeError('"image" must be numpy.ndarray')
    
    # copy of matrix
    matConf = matrix.astype(float).copy()
    
    for i in range(n):
        matConf[i,:] = (matConf[i,:]/ni[i])*wi[i]
    
    # user's accuracy
    ua = np.diag(matConf)/np.sum(matConf, axis = 1)
    
    # total Wi
    total_wi = np.sum(matConf, axis = 1)
    
    # copy the matrix of proportions
    mat_conf = matConf.copy()
    # building the matrix of proportion area
    mat_conf = np.concatenate([mat_conf, total_wi.reshape(n, 1)], axis = 1)
    mat_conf = np.concatenate([mat_conf, pixels.reshape(n, 1)], axis = 1)
    # total Wi in rows
    total = np.sum(mat_conf, axis = 0)
    # final matrix
    mat_conf = np.concatenate([mat_conf, total.reshape(1, n+2)], axis = 0)
    
    namesCol = []
    for i in np.arange(1, n + 1): namesCol.append(str(i))
    namesCol.extend(['Total[Wi]', 'Area[pixels]'])

    namesRow = []
    for i in np.arange(1, n + 1): namesRow.append(str(i))
    namesRow.extend(['Total'])

    # error matrix (DataFrame) in proportions of area
    cm_prop_area = pd.DataFrame(np.round(mat_conf, 6), columns = namesCol, index = namesRow)
    
    # overall accuracy
    oa = np.diag(matConf).sum()/np.sum(matConf)
    
    # confidence interval for Overall accuracy at 95% 1.96
    conf_int_oa = list(map(lambda Wi, UA, Ni: (Wi)**2*UA*(1-UA)/(Ni-1), wi, ua, ni))
    conf_int_oa = conf*np.sqrt(np.nansum((np.array(conf_int_oa))))
        
    # confidence interval for user's accuracy at 95% 1.96
    conf_int_ua = conf*np.sqrt(np.array(list(map(lambda UA, Ni: UA*(1-UA)/(Ni-1), ua, ni))))
    conf_int_ua[np.isnan(conf_int_ua)] = 0
    
    # confidence interval for the area at 95%
    sp = []
    for i in np.arange(n):
        s_pi = list(map(lambda Wi, Pik, Ni: (Wi*Pik - Pik**2)/(Ni-1), wi, matConf[:,i], ni))
        s_pi = np.sqrt(np.nansum(np.array(s_pi)))
        sp.append(s_pi)
    
    # S(A)=1.96*s(p)*A(total) in ha
    SA = conf*np.array(sp)*(np.array(pixels).sum())*(pixel_size**2/10000)
    
    # Area total estimated
    A = total[0:n]*(np.array(pixels).sum())*(pixel_size**2/10000)
    
    result = {'Overall_accuracy':oa,
              'CM_estimatedPA':cm_prop_area,
              'Users_accuracy':ua,
              'Users_accuracy_95%':conf_int_ua,
              'Area_total_estimated': A,
              'Area_at_95%': SA,
              'conf_int_oa': conf_int_oa,
              'iclass': iclass
             }
    
    return result


def print_info(params):
    
    ''' Information: Confusion Matrix by Estimated Proportions of area an uncertainty
    
    Parameters:
    
        params: ```confintervalML``` result. See the function: https://github.com/ytarazona/scikit-eo/blob/main/scikeo/process.py
    
    Return:
        
        Information of confusion matrix by proportions of area, overall accuracy, user's accuracy with confidence interval 
        and estimated area with confidence interval as well.
        
    Note:
        This function was tested using ground-truth values obtained by Olofsson et al. (2014).
        
    '''
    oa = params.get('Overall_accuracy')
    matrixCEA = params.get('CM_estimatedPA')
    a = params.get('Users_accuracy')
    b = params.get('Users_accuracy_95%')
    c = params.get('Area_total_estimated')
    d = params.get('Area_at_95%')
    e = params.get('conf_int_oa')
    f = params.get('iclass')
    
    print('***** Confusion Matrix by Estimated Proportions of area an uncertainty*****')
    print('')
    print('Overall accuracy with 95%')
    print(f'{oa:.6f} ± {e:.6f}')
    print('')
    print('Confusion matrix')
    print(matrixCEA)
    print('')
    print('User´s accuracy with 95%')
    for i in range(b.shape[0]):
        print(f'{f[i]}: {a[i]:.6f} ± {b[i]:.6f}')
    print('')
    print('Estimating area (Ha) and uncertainty with 95%')
    for i in range(b.shape[0]):
        print(f'{f[i]}: {c[i]:.6f} ± {d[i]:.6f}')

In [31]:
img = np.array([200000,150000,3200000,6450000])

In [32]:
values = np.array(([66,0,5,4],[0,55,8,12],[1,0,153,11],[2,1,9,313]))
values.shape

(4, 4)

In [33]:
result = confintervalML(matrix = values, image_pred = img, pixel_size = 30, nodata = -9999)

In [34]:
print_info(result)

***** Confusion Matrix by Estimated Proportions of area an uncertainty*****

Overall accuracy with 95%
0.946512 ± 0.018484

Confusion matrix
              1         2         3         4  Total[Wi]  Area[pixels]
1      0.017600  0.000000  0.001333  0.001067      0.020      200000.0
2      0.000000  0.011000  0.001600  0.002400      0.015      150000.0
3      0.001939  0.000000  0.296727  0.021333      0.320     3200000.0
4      0.003969  0.001985  0.017862  0.621185      0.645     6450000.0
Total  0.023509  0.012985  0.317522  0.645985      1.000    10000000.0

User´s accuracy with 95%
0: 0.880000 ± 0.074041
1: 0.733333 ± 0.100757
2: 0.927273 ± 0.039745
3: 0.963077 ± 0.020534

Estimating area (Ha) and uncertainty with 95%
0: 21157.762238 ± 6157.634386
1: 11686.153846 ± 3755.826025
2: 285769.930070 ± 15509.836298
3: 581386.153846 ± 16281.656352
